# Results 3    
The structure of our photos is quite simple.   
Each photo of the catalogue is divided into tiles on a 4x5 grid, the useful information is to be found in two tiles, (1,2) and (2,2).
The same kind of heuristic is used to isolate information with queries.    
Thus each catalogue item and query give 2 feature vectors.  
For a query each catalogue item is ranked according the best matcher among these 4 feature vectors.    
It gives a **71%** 10-accuracy result.

In [1]:
import os
import re

import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import io
import IPython.display as display
from PIL import Image
to_ignore = [53,89,110,120,127,131,144,167,159,190]
sess = tf.Session()

## Load Dataset
### Catalogue Images

In [2]:
cat_dir = 'db/robes/cat/'
list_cat = [cat_dir+f for f in os.listdir(cat_dir) if re.search('jpg|JPG', f)]
list_cat = list(filter(lambda x: "_0" in x, list_cat))
list_cat.sort(key=lambda x: int(x.split("/")[-1].split("_")[0]))
print(str(len(list_cat))+" items in the catalogue")

210 items in the catalogue


In [3]:
### Query Images
query_dir = 'db/robes/mod/'
list_query = [query_dir+f for f in os.listdir(query_dir) if re.search('jpg|JPG', f)]
list_query.sort(key=lambda x: int(x.split("/")[-1].split(".")[0]))
print(str(len(list_query))+" queries to perform")

210 queries to perform


## Load Feature Extractor

In [4]:
model_dir = 'models'
def create_graph():
    with gfile.FastGFile(os.path.join(model_dir, 'classify_image_graph_def.pb'), 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')
create_graph()
sess = tf.Session()

## Feature Extraction Routine

In [25]:
#This routine operates the cropings before extracting features
def extract_features(list_images,mod=True,p=False):
    nb_features = 2048
    features = []

    next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')

    for ind, image in enumerate(list_images):
        #Getting rid of ignored
        if ind in to_ignore:
            mini_features = []
            
            for i in range(2):
                mini_features.append(np.zeros(nb_features))
            features.append(mini_features[:])
            if p:
                for (i,x) in enumerate(features[-1]):
                    print(i,x)
            continue
        if (ind%1 == 0) and p:
            print('Processing %s...' % (image))
        if not gfile.Exists(image):
            tf.logging.fatal('File does not exist %s', image)

        image_data = gfile.FastGFile(image, 'rb').read()
        image_data_tensor = tf.image.decode_jpeg(image_data)
        
        #Getting all the crops
        mini_features = []
        
        if mod:
            of_y = int(sess.run(image_data_tensor).shape[0]/10)
            of_x = int(sess.run(image_data_tensor).shape[1]/11)
            for i in range(10):
                if i in [0,1,2,3,6,7,8,9]:
                    continue
                for j in range(11):
                    if j != 5:
                        continue
        
                    image_data_croped  = tf.image.crop_to_bounding_box(image_data_tensor,i*of_y,j*of_x,of_y,of_x)
                    image_data_modified = tf.image.encode_jpeg(image_data_croped)
                    predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': sess.run(image_data_modified)})
                    mini_features.append(np.squeeze(predictions))
        else:
            of_y = int(sess.run(image_data_tensor).shape[0]/4)
            of_x = int(sess.run(image_data_tensor).shape[1]/5)
        
            for i in range(4):
                if i == 0 or i == 3:
                    continue
                for j in range(5):
                    if j == 0 or j == 4 or j == 1 or j == 3:
                        continue
                    image_data_croped  = tf.image.crop_to_bounding_box(image_data_tensor,i*of_y,j*of_x,of_y,of_x)
                    image_data_modified = tf.image.encode_jpeg(image_data_croped)
                    predictions = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': sess.run(image_data_modified)})
                    mini_features.append(np.squeeze(predictions))
        
        features.append(mini_features[:])
        if p:
            for (i,x) in enumerate(features[-1]):
                print(i,x)
    print("Done")
    return features

## Computing Features

In [21]:
cat_features = extract_features(list_cat,mod=False,p=True)

Processing db/robes/cat/0_0.jpg...
0 [ 0.23089719  0.01549842  0.04701736 ...,  0.24392782  0.03403777
  0.0012143 ]
1 [ 0.51083297  0.          0.11807527 ...,  0.13151902  0.0203489
  0.06101101]
Processing db/robes/cat/1_0.jpg...
0 [ 0.79108655  0.          0.20937654 ...,  0.07683647  0.00784178
  0.1895777 ]
1 [ 0.64674944  0.06291676  0.30947843 ...,  0.00100118  0.03815445
  0.12339684]
Processing db/robes/cat/2_0.jpg...
0 [ 0.7268346   0.01163832  0.48327768 ...,  0.224869    0.14149082
  0.62228531]
1 [ 0.57509995  0.01561046  0.38554186 ...,  0.09301012  0.09681492
  0.67719191]
Processing db/robes/cat/3_0.jpg...
0 [ 0.58180779  0.03993735  0.19787343 ...,  0.2793707   0.08866801
  0.15142497]
1 [ 1.22035408  0.01921894  0.05327503 ...,  0.0529275   0.09201792
  0.25277239]
Processing db/robes/cat/4_0.jpg...
0 [ 0.28511822  0.32348004  0.15528804 ...,  0.00138841  0.06939041
  0.21097586]
1 [ 0.12571374  0.40662625  0.11578248 ...,  0.15336122  0.          0.0541094 ]
Process

In [26]:
query_features = extract_features(list_query,mod=True,p=True)

Processing db/robes/mod/0.jpg...
0 [ 0.16498202  0.02549706  0.3572368  ...,  0.30864489  0.17487794
  0.0985774 ]
1 [ 0.21631058  0.          0.18364193 ...,  0.44218335  0.1266522
  0.03936384]
Processing db/robes/mod/1.jpg...
0 [ 0.24344102  0.0098957   0.23656125 ...,  0.37234509  0.01572444
  0.60094982]
1 [ 0.0874515   0.          0.07648849 ...,  0.32660639  0.0134766
  0.07581992]
Processing db/robes/mod/2.jpg...
0 [ 0.06782524  0.02133217  0.14569598 ...,  0.48883528  0.          0.10451044]
1 [ 0.00175978  0.0027778   0.28049174 ...,  0.48344392  0.00865945
  0.29762012]
Processing db/robes/mod/3.jpg...
0 [ 0.78903639  0.25527132  0.22778207 ...,  0.26403049  0.10436608
  0.77165782]
1 [ 1.12414467  0.57847935  0.01943058 ...,  0.3929306   0.13069746
  0.97615969]
Processing db/robes/mod/4.jpg...
0 [ 0.02317904  0.29355073  0.33249259 ...,  0.00501774  0.          0.40208191]
1 [ 0.06468473  0.29872909  0.2866866  ...,  0.11832022  0.          0.23616438]
Processing db/robes/

In [27]:
#Safety check
len(cat_features),len(query_features)

(210, 210)

## Computing Results
For each queries, the best matching items in the catalogue, according to cosine similarity, are found.

In [28]:
#Cosine Similarity
def sim(vecA,vecB):
    return vecA.dot(vecB)
#Perform the best matching retrieval
#For a given query and a given object the matching score is the max
#of similiraties over the 4 corresponding vectors in memory.
#Accuracy: what it means to match, by default it is to be in the top 10 closest
#p: print debug
def query(i_query,accuracy=10,p=False):
    sim_vec = []
    for i in range(len(cat_features)):
        M = 0.0
        for k in range(2):
            for l in range(2):
                M = max(M,sim(query_features[i_query][k],cat_features[i][l]))
        sim_vec.append(M)
    sim_vec = np.array(sim_vec)
    arg_s = sim_vec.argsort()[:-accuracy:-1]
    if p:
        print(i_query,arg_s,[sim_vec[i] for i in arg_s], sim_vec[i_query])
    return i_query in arg_s

In [29]:
#Normalization before querying
for i in range(len(cat_features)):
    if not i in to_ignore:
        for j in range(2):
            cat_features[i][j] /= np.linalg.norm(cat_features[i][j])
            query_features[i][j] /= np.linalg.norm(query_features[i][j])

In [30]:
#Getting result while getting rid of ignored val
matching_frac = 0
ignored = 0
for i in range(len(cat_features)):
    if not i in to_ignore:
        matching_frac += query(i)
    else:
        ignored += 1
matching_frac /= (len(cat_features)-ignored)

# Result

In [31]:
print("This method gives a "+str(matching_frac*100)+" 10-accuracy success")

This method gives a 71.0 10-accuracy success


# Conclusion and Perspectives
This **hand-made** heuristic to crop the images of this dataset can be justified in our application setting:   
-The retailer can make the relevant crops when it builds its catalogue    
-The user can zoom over what interests him when he make the query    
However this example emphasizes **the importance of selecting relevant part of the clothes**, wether it is done by hand or automatically.   
Methods in the spirit of "Points of Interets" as used in SIFT descriptors could be used to get rid of this hand-crafted dataset
specific part of the method.    
Enventually to handle more crops efficiently in memory, techniques such as **memory vectors** could be used.    